<a href="https://colab.research.google.com/github/vinodwij/dsgp-group23-business-data-harbour/blob/uzama-a/Sentiment_Analysis_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import numpy as np
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

In [27]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
# printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [29]:
#loading the data from csv file to pandas dataframe
review_data = pd.read_csv('/content/Watch Review dataset.csv', encoding = 'ISO-8859-1')

In [30]:
#checking the number of rows and columns
review_data.shape

(2125, 82)

In [31]:
review_data = review_data[['product_title', 'review_rating', 'review_text']]

In [32]:
#counting the number of missing values in the
review_data.isnull().sum()

product_title    180
review_rating    180
review_text      185
dtype: int64

In [33]:
#removing the rows that contain null values
review_data = review_data.dropna()
review_data

,product_title,review_rating,review_text
0,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,"bought it as a replacement, turns out the wris..."
1,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,CASIO WAS ONCE A TOP OF THE LINE WATCH. DEPEND...
2,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Everything works the way it should but it is tiny
3,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Small. For Kids wrist not adults ..
4,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Did not read the spes. Carefully just satisfie...
...,...,...,...
1941,Casio Men's 'Vintage' Quartz Metal and Resin C...,3.0,It was smaller than expected
1942,Casio Men's 'Vintage' Quartz Metal and Resin C...,3.0,These were better made in the 80's. For all th...
1943,Casio Men's 'Vintage' Quartz Metal and Resin C...,3.0,Light doesnât illuminate very well. Watch ba...
1944,Casio Men's 'Vintage' Quartz Metal and Resin C...,3.0,The watch certainly keeps very good time but t...


In [34]:
review_data['ID'] =  review_data.index + 1

<ipython-input-34-17e28e3ac304>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_data['ID'] =  review_data.index + 1


In [35]:
review_data.head()

,product_title,review_rating,review_text,ID
0,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,"bought it as a replacement, turns out the wris...",1
1,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,CASIO WAS ONCE A TOP OF THE LINE WATCH. DEPEND...,2
2,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Everything works the way it should but it is tiny,3
3,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Small. For Kids wrist not adults ..,4
4,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Did not read the spes. Carefully just satisfie...,5


In [36]:
#counting the number of missing values in the
review_data.isnull().sum()

product_title    0
review_rating    0
review_text      0
ID               0
dtype: int64

In [37]:
#checking the distribution of the target column
review_data['review_rating'].value_counts()

3.0    511
5.0    400
1.0    363
4.0    353
2.0    313
Name: review_rating, dtype: int64

In [38]:
review_points = review_data['review_rating'].tolist()

In [39]:
review_label = []
for i in review_points:
  if i == 1 or i == 2:
    i = 0
  elif i == 3:
    i = 3
  else:
    i = 1
  review_label.append(i)

In [40]:
review_label

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [41]:
review_data['review_label'] = review_label

In [42]:
review_data

,product_title,review_rating,review_text,ID,review_label
0,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,"bought it as a replacement, turns out the wris...",1,0
1,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,CASIO WAS ONCE A TOP OF THE LINE WATCH. DEPEND...,2,0
2,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Everything works the way it should but it is tiny,3,0
3,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Small. For Kids wrist not adults ..,4,0
4,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Did not read the spes. Carefully just satisfie...,5,0
...,...,...,...,...,...
1941,Casio Men's 'Vintage' Quartz Metal and Resin C...,3.0,It was smaller than expected,1942,3
1942,Casio Men's 'Vintage' Quartz Metal and Resin C...,3.0,These were better made in the 80's. For all th...,1943,3
1943,Casio Men's 'Vintage' Quartz Metal and Resin C...,3.0,Light doesnât illuminate very well. Watch ba...,1944,3
1944,Casio Men's 'Vintage' Quartz Metal and Resin C...,3.0,The watch certainly keeps very good time but t...,1945,3


In [43]:
#checking the distribution of the target column
review_data['review_label'].value_counts()

1    753
0    676
3    511
Name: review_label, dtype: int64

stemming is the process of of reducing a word to its root word



In [44]:
port_stem = PorterStemmer()

In [45]:
def stemming(content):
    if not isinstance(content, str):
        # Convert non-string content to string
        content = str(content)
    # Remove everything other than letters in a text
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    # Convert every letter to lowercase
    stemmed_content = stemmed_content.lower()
    # Split the words and add to a list
    stemmed_content = stemmed_content.split()
    # Stem the words, but do not stem stopwords because they have no meaning
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    # Join the stemmed words
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content


In [46]:
review_data['stemmed_text'] = review_data['review_text'].apply(stemming)

In [47]:
review_data.head()

,product_title,review_rating,review_text,ID,review_label,stemmed_text
0,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,"bought it as a replacement, turns out the wris...",1,0,bought replac turn wrist band child size bewar
1,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,CASIO WAS ONCE A TOP OF THE LINE WATCH. DEPEND...,2,0,casio top line watch depend like timex ad new ...
2,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Everything works the way it should but it is tiny,3,0,everyth work way tini
3,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Small. For Kids wrist not adults ..,4,0,small kid wrist adult
4,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Did not read the spes. Carefully just satisfie...,5,0,read spe care satisfi price


In [48]:
#separating the data and label
X = review_data['stemmed_text'].values
Y = review_data['review_label'].values

In [49]:
print(X)

['bought replac turn wrist band child size bewar'
 'casio top line watch depend like timex ad new twist class style wow dissapoint cheapli made look like made child'
 'everyth work way tini' ... 'light illumin well watch band need larger'
 'watch certainli keep good time end light bright enough use alarm quiet hear signific ambient nois biggest problem howev length band small feel normal size wrist watch tight even last hole addit band short last hole two tail band remain retain stick hang sleev shirt band need inch longer small wrist watch would fine care light alarm'
 'work howev look feel like dollar watch great watch kid']


In [50]:
print(Y)

[0 0 0 ... 3 3 3]


Spiltting the data to training data and test data

In [51]:
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size = 0.2, stratify = Y, random_state = 2)

In [52]:
print(X.shape, X_train.shape, X_test.shape)

(1940,) (1552,) (388,)


In [53]:
print(X_train)

['timex expedit scout indiglo cream dial timex low cost area lot timex fast ship receiv watch typic timex packag small instruct print expect indiglo featur wear well wrist fairli easi read adequ light date hard read band soft comfort prefer nato bond comfort box clasp keeper seem timex typic crown easi posit inward outward decent grip easi turn adjust time date good fit finish watch price point loos minut per hour reset hack second next day lost minut store differ posit time mayb later swap fresh batteri work trash goe second hand hold watch upright face second hand close marker close fall mark half way back mark seem second slop backlash ye saw good bad review decid want take chanc make model loos minut per hour disappoint grew timex watch sinc seiko orient casio want add timex current herd sake timex wrist simplist basic cream dial timex ticker guess lucki time look someth els know timex watch hold time well get time'
 'love color face us lot smaller advertis okay work outfit match w

In [54]:
print(X_test)

['analog timex watch year issu one nice howev indiglo stop work coupl week two cheap timex watch year old still work fine edit look fix indiglo stem suppos tini e clip press indiglo switch push crown clip must come dislodg somehow open back watch e clip tini could anywher got anoth e clip old timex put watch tweezer indigo work'
 'got earli packag nice n nice watch'
 'tinker wife know love watch told love met year ago doubt gift default watch rolex movado patek philipp breitl omega last gift th birthday chopard also bought seiko year ago reason background brag watch collect fact told like watch meant like wear seiko timex speed wore seiko yard work rebuild engin tinker hood antiqu car son help one day garag said dad wear watch bang around hood truck almost fell last seiko idea anywher near expens immedi went amazon bought littl jewel wear routin tinker also wear work offic set swim even went scuba dive instead trusti rolex submarin first gift wife timex rate meter went meter ft fine co

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming X_train and X_test are numpy arrays of strings
vectorizer = TfidfVectorizer()

# Use numpy arrays directly
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [56]:
print(X_train)

  (0, 1481)	0.03525996558533989
  (0, 1896)	0.04776268491825824
  (0, 1131)	0.059762289181560804
  (0, 3230)	0.05079884731632706
  (0, 2018)	0.03210662350552287
  (0, 2041)	0.08758012011758692
  (0, 1562)	0.05722862808368279
  (0, 3537)	0.09247859699130985
  (0, 307)	0.059762289181560804
  (0, 3147)	0.09247859699130985
  (0, 2986)	0.08758012011758692
  (0, 1630)	0.09247859699130985
  (0, 816)	0.06813629070761411
  (0, 37)	0.06528044703722274
  (0, 530)	0.04089582501683559
  (0, 2417)	0.08410459546525655
  (0, 3042)	0.07573059393920324
  (0, 3150)	0.05311710761847477
  (0, 1545)	0.09247859699130985
  (0, 993)	0.048692461297641286
  (0, 2211)	0.0517978567245736
  (0, 2070)	0.04301428612945418
  (0, 557)	0.07920611859153363
  (0, 3449)	0.04791276300317711
  (0, 3829)	0.08769559602333257
  :	:
  (1550, 1510)	0.2623595877999466
  (1550, 3799)	0.2623595877999466
  (1550, 968)	0.24846272500954028
  (1550, 866)	0.2386027439453851
  (1550, 486)	0.21080901836457244
  (1550, 1176)	0.2247058811549

In [57]:
print(X_test)

  (0, 3955)	0.1456916821346206
  (0, 3920)	0.167039030594757
  (0, 3860)	0.08419226315815354
  (0, 3839)	0.12509818204759782
  (0, 3656)	0.08393096348505619
  (0, 3553)	0.21362691976681833
  (0, 3552)	0.25581226403937096
  (0, 3432)	0.10922686860071804
  (0, 3407)	0.10758793272964584
  (0, 3329)	0.08887862539406079
  (0, 3322)	0.07478314477780543
  (0, 3315)	0.1435009050224204
  (0, 3228)	0.1435009050224204
  (0, 2734)	0.09578352188674126
  (0, 2731)	0.09869198752243707
  (0, 2649)	0.09578352188674126
  (0, 2401)	0.10681345988340916
  (0, 2396)	0.05717848211423371
  (0, 2389)	0.18261028632904855
  (0, 2306)	0.07253942346170679
  (0, 2259)	0.12485722379786869
  (0, 2018)	0.056595002980316424
  (0, 1824)	0.08730044749908766
  (0, 1761)	0.12671018816574214
  (0, 1759)	0.2887245319164079
  :	:
  (386, 1454)	0.1068870593837957
  (386, 1345)	0.1113836728322757
  (386, 1333)	0.1633495571186587
  (386, 1329)	0.1617989455872016
  (386, 1160)	0.12663655096927764
  (386, 977)	0.14239551187733532


Hnadle Imbalance Dataset

In [67]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_train_smote, Y_train_smote = smote.fit_resample(X_train, Y_train)
print(X_train_smote.shape, Y_train_smote.shape)

(1806, 3971) (1806,)


In [68]:
# Assuming Y_train_smote is a numpy array
unique_elements, counts_elements = np.unique(Y_train_smote, return_counts=True)

# Print the counts of unique elements
for element, count in zip(unique_elements, counts_elements):
    print(f"{element}: {count}")


0: 602
1: 602
3: 602
